In [1]:
import pandas as pd 
import json
import numpy as np
import datetime
import time
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.metrics import f1_score,accuracy_score,recall_score,roc_auc_score
from sklearn import tree
import scikitplot as skplt

In [2]:
# bd_3 = pd.read_excel(r'F://svn/新建文件夹 (2)//通用分项目/0_基础信息/百度进件数据/百度3月进件.xlsx')
# bd_4 = pd.read_excel(r'F://svn/新建文件夹 (2)//通用分项目/0_基础信息/百度进件数据/百度4月进件.xlsx')
# bd_5 = pd.read_excel(r'F://svn/新建文件夹 (2)//通用分项目/0_基础信息/百度进件数据/百度5月进件.xlsx')
# bd_6 = pd.read_excel(r'F://svn/新建文件夹 (2)//通用分项目/0_基础信息/百度进件数据/百度6月进件.xlsx')
# bd_7 = pd.read_excel(r'F://svn/新建文件夹 (2)//通用分项目/0_基础信息/百度进件数据/百度7月份进件.xlsx')
# bd_8 = pd.read_excel(r'F://svn/新建文件夹 (2)//通用分项目/0_基础信息/百度进件数据/百度8月份进件.xlsx')

In [3]:
# bd =pd.concat([bd_3,bd_4,bd_5,bd_6,bd_7,bd_8],axis=0)

In [4]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

from numpy import log
import datetime
from dateutil.relativedelta import relativedelta

class IV:
    # func: calcWOE
    # input:
    #       dataset: 数据集
    #       col:  需要观测的特征
    #       targe: 标签
    # output:
    #       该标签的WOE结果集 
    def calcWOE(dataset,col,targe):
        subdata=pd.DataFrame(dataset.groupby(col)[col].count())
        suby=pd.DataFrame(dataset.groupby(col)[targe].sum())
        data=pd.DataFrame(pd.merge(subdata,suby,how="left",left_index=True,right_index=True))
        b_total=data[targe].sum()
        total=data[col].sum()
        g_total=total-b_total
        data["bad"]=data.apply(lambda x:round(x[targe]/b_total,11),axis=1)
        data["good"]=data.apply(lambda x:round((x[col]-x[targe])/g_total,11),axis=1)
        data["WOE"]=data.apply(lambda x:log(x.bad/x.good),axis=1)
        return data.loc[:,["bad","good","WOE"]]
    
    # func: calcIV
    # input: 
    #       dataset: 该标签的WOE结果集 
    # output
    #       特征IV值
    def calcIV(dataset):
        dataset["IV"]=dataset.apply(lambda x:(x.bad-x.good)*x.WOE,axis=1)
#         print(dataset)
        IV=sum(dataset["IV"])
        return IV
    
    # func:IV
    # 主函数
    # input：
    #       data; 数据集，
    #       target:  标签名（标签数据需要在数据内）
    #       all_feature:  需要进行计算IV值的特征集合
    #       binning: 是否分箱
    # output:
    #       无,打印输出
    def IV(data,target,all_feature=[],check=False,binning=True):
        # 分箱
        # 如果未分箱，要进行分箱默认只有四等分
        if all_feature==[]: 
            all_feature = data.columns.values.tolist()[2:]
        for feature in all_feature:
            try:
                if not binning:
                    data[feature] = pd.cut(data[feature],4)
                # 去除其中WOE值为无穷大的 
                data_WOE=(IV.calcWOE(data,feature,target).fillna(0)).query('WOE!=inf&WOE!=-inf')
                if check:
#                     print(data_WOE)
#                     print('------')
                    print(pd.DataFrame(IV.calcWOE(data,feature,target).fillna(0)))
                data_IV=IV.calcIV(data_WOE)
                print("{0}\t\t:{1}".format(feature,data_IV))
            except:
                print("{0} has some error".format(feature))
        return pd.DataFrame(IV.calcWOE(data,feature,target).fillna(0))
        # if(data_IV==0.0):
        #     print(data_WOE)


class PSI:
    # 含两个版本的PSI计算方式
    # 分为非loop版本 和 loop版本
    # loop 版本在处理批量数据上的处理熟读会优于非loop版本
    # 大概200的特征处理后的数据能优化20倍, 非loop版本会多次统计处理数据表
    def binning(data,col,_type=1,X=10):
        # func: binning
        # input: data -- 数据集
        # input: col -- 需要进行分箱的特征
        # input: _type -- 用什么办法进行分箱
        # input: X  -- 分箱数量
        try:
            mark = False
            if data[col].value_counts().shape[0]>10:            
                if _type == 1:# 等频
                    data[col+'_newbin'] = pd.qcut(data[col],X,labels=False,duplicates='drop')
                if _type == 2:# 等宽
                    data[col+'_newbin'] = pd.cut(data[col],X,labels=False)
                if _type == 3:# 等频 速度会比较慢
                    edages = pd.Series([i/n for i in range(n)])
                    data[col+'_newbin'] = data[col].rank(pct=1).astype(float).apply(lambda x: (x<=edages).idxmax())
                mark = True
            return data,mark
        except:
            print("{0} 不是连续性变量，不做分箱处理 ".format(col))
            return data,False
        
    def get_base_exp_data(data,time_col, base_year, base_month,gap,target):
        # func:get_base_exp_data
        # input:
        #       _data: 数据集
        #       time_col:时间变量
        #       base_year: 基准时间-年
        #       base_month: 基准时间-月
        #       gap: 间隔时间
        #       target: 目标变量
        # output:
        #       目标变量的计算基础数据
        data['base_year'] = base_year
        data['base_month'] = base_month
        data['exp_month'] = base_month+gap     
        if df['exp_month'].values[1]<=12:  # 只看一列中的第一个
            data['exp_year']  = base_year
        else:
            data['exp_year']  = base_year+1
            data['exp_month'] = base_month+gap-12
        data[time_col+'_year'] = data[time_col].apply(lambda x:x.year)
        data[time_col+'_month'] = data[time_col].apply(lambda x:x.month)
        data_base = data.query(time_col+'_year==base_year&'+time_col+'_month==base_month')
        data_exp  = data.query(time_col+'_year==exp_year&'+time_col+'_month==exp_month')
        return data_base.loc[:,[target]],data_exp.loc[:,[target]]
    
    def get_base_exp_data_loop(data,time_col, base_year, base_month,gap):
        # func:get_base_exp_data
        # input:
        #       _data: 数据集
        #       time_col:时间变量
        #       base_year: 基准时间-年
        #       base_month: 基准时间-月
        #       gap: 间隔时间
        # output:
        #       全量的计算基础数据
        data['base_year'] = base_year
        data['base_month'] = base_month

        data['exp_month'] = base_month+gap  
        #print(data['exp_month'][0])
        
        if df['exp_month'].values[1]<=12:  # 只看一列中的第一个
            data['exp_year']  = base_year
        else:
            data['exp_year']  = base_year+1
            data['exp_month'] = base_month+gap-12
        data[time_col+'_year'] = data[time_col].apply(lambda x:x.year)
        data[time_col+'_month'] = data[time_col].apply(lambda x:x.month)
        data_base = data.query(time_col+'_year==base_year&'+time_col+'_month==base_month')
        data_exp  = data.query(time_col+'_year==exp_year&'+time_col+'_month==exp_month')
        return data_base.loc[:],data_exp.loc[:]
    
    def get_base_exp_data_loop_part(data):
        # func:get_base_exp_data
        # input:
        #       _data: 数据集
        #       time_col:时间变量
        #       base_year: 基准时间-年
        #       base_month: 基准时间-月
        #       gap: 间隔时间
        # output:
        #       全量的计算基础数据
#         data['base_year'] = base_year
#         data['base_month'] = base_month

#         data['exp_month'] = base_month+gap  
        #print(data['exp_month'][0])
        
#         if df['exp_month'].values[1]<=12:  # 只看一列中的第一个
#             data['exp_year']  = base_year
#         else:
#             data['exp_year']  = base_year+1
#             data['exp_month'] = base_month+gap-12
#         data[time_col+'_year'] = data[time_col].apply(lambda x:x.year)
#         data[time_col+'_month'] = data[time_col].apply(lambda x:x.month)
#         data_base = data.query(time_col+'_year==base_year&'+time_col+'_month==base_month')
#         data_exp  = data.query(time_col+'_year==exp_year&'+time_col+'_month==exp_month')
        data_base = data.query('tag==0|tag==1').query('CREATED_DATE<"2020-07-01"')
        data_exp = data.query('tag==0|tag==1').query('CREATED_DATE>"2020-08-01"')
        return data_base.loc[:],data_exp.loc[:]
    
    def PSI(data, time_col, base_year, base_month, gap,target,ignore_inf=True,test=False):
        # input
        # base_time = year+month like '2019-06-01'
        data, mark = PSI.binning(data,target,1,8)
        if mark: target = target+'_newbin'
        data_base, data_exp = PSI.get_base_exp_data(data, time_col,base_year, base_month, gap, target)
        if test: print(data_base,data_exp)
        # 分箱统计
        result_base = pd.DataFrame(data_base[target].value_counts())
        result_base['base_p'] = result_base[target].apply(lambda x: x/result_base[target].sum())
        result_exp  = pd.DataFrame(data_exp[target].value_counts())
        result_exp['exp_p'] = result_exp[target].apply(lambda x: x/result_exp[target].sum())
        result =pd.DataFrame(pd.merge(result_base,result_exp,how="left",left_index=True,right_index=True))
        result = result.loc[:,['base_p','exp_p']]
        result = result.fillna(0)
        length = result.shape[0]
        if test: print(result)
        if test: print(result.query('base_p==0'))
        psi = 0 
        result['psi'] = (result['exp_p'].apply(lambda x:x)-result['base_p'].apply(lambda x:x))*log(result['exp_p'].apply(lambda x:x)/result['base_p'].apply(lambda x:x))
        if test: print(result['psi'])
        print('{0}\t{1}'.format(target,result.query('psi!=inf')['psi'].sum())) if ignore_inf else print('{0}\t{1}'.format(target,result['psi'].sum()))
    
    def PSI_loop_part(data, targets, bin=True, ignore_inf=True, test=False):
        if bin:
            bin_list = []
            bin_listadd = []
            for target in targets:
                data, mark = PSI.binning(data,target,1,8)
                if mark: 
                    bin_list.append(target)
                    bin_listadd.append(target+'_newbin')        
            targets = list((set(targets)-set(bin_list))|set(bin_listadd))
        
        data_base, data_exp = PSI.get_base_exp_data_loop_part(data)
        for target in targets:
            result_base = pd.DataFrame(data_base[target].value_counts())
            result_base['base_p'] = result_base[target].apply(lambda x: x/result_base[target].sum())
            result_exp  = pd.DataFrame(data_exp[target].value_counts())
            result_exp['exp_p'] = result_exp[target].apply(lambda x: x/result_exp[target].sum())
            result =pd.DataFrame(pd.merge(result_base,result_exp,how="left",left_index=True,right_index=True))
            result = result.loc[:,['base_p','exp_p']]
            result = result.fillna(0)
            length = result.shape[0]
            if test: print(result)
            if test: print(result.query('base_p==0'))
            psi = 0 
            result['psi'] = (result['exp_p'].apply(lambda x:x)-result['base_p'].apply(lambda x:x))*log(result['exp_p'].apply(lambda x:x)/result['base_p'].apply(lambda x:x))
            if test: print(result['psi'])
            print('{0}\t{1}'.format(target,result.query('psi!=inf')['psi'].sum())) if ignore_inf else print('{0}\t{1}'.format(target,result['psi'].sum()))
    



In [5]:
import pandas as pd
import numpy as np
from scipy.stats import chi2

def Chi2(df, total_col, bad_col,overallRate):
    '''
     #此函数计算卡方值
     :df dataFrame
     :total_col 每个值得总数量
     :bad_col 每个值的坏数据数量
     :overallRate 坏数据的占比
     : return 卡方值
    '''
    df2=df.copy()
    df2['expected']=df[total_col].apply(lambda x: x*overallRate)
    combined=zip(df2['expected'], df2[bad_col])
    chi=[(i[0]-i[1])**2/i[0] for i in combined]
    chi2=sum(chi)
    return chi2

def chi22(arr):
    '''
    计算卡方值
    arr：频数统计表，二维numpy数组
    '''
    assert(arr.ndim == 2) 
    # assert断言是声明其布尔值必须为真的判定，如果发生异常就说明表达示为假。
    # 可以理解assert断言语句为raise-if-not，用来测试表示式，其返回值为假，就会触发异常。
    
    R_N = arr.sum(axis=1) # 计算每行总频数
    C_N = arr.sum(axis=0) # 计算每列总频数
    N = arr.sum() # 总频数
    
    E = np.ones(arr.shape)*C_N/N # 计算期望频数
    E = (E.T*R_N).T
    square = (arr-E)**2/E
    square[E==0] = 0 # 当期望频数为0时，作为分母无意义，不计入卡方值
    
    v = square.sum() # 卡方值
    return v

def chiMerge(df,col,target,max_groups = None,threshold = None):
    '''
    卡方分箱
    df: pandas DataFrame 数据集
    col: 需要分箱的变量名（数值型）/score
    target：类标签 mark
    max_groups: 最大分组数 N
    threshold： 卡方阈值，如果未指定max_groups，默认使用置信度95%设置threshold
    return：包括各组的起始值的列表
    '''
    freq_tab = pd.crosstab(df[col],df[target])
    
    # 转成numpy数组用于计算
    freq = freq_tab.values
    
    # 初始分组切分点，每个变量值都是切分点，每组中只包含一个变量值
    # 分组区间是左闭右开的，如cutoffs = [1,2,3]，表示区间，[1,2),[2,3).[3,3+)
    cutoffs = freq_tab.index.values
    
    # 如果没有指定最大分组
    if max_groups is None:
        # 如果没有指定卡方阈值，就以95%的置信度（自由度为类数目-1）设定阈值
        if threshold is None:
            # 类数目
            cls_num = freq.shape[-1]
            threshold = chi2.isf(0.05,df = cls_num-1)
    
    while True:
        minvalue = None
        minidx = None
        # 从第一组开始，依次计算两组卡方值，并判断是否小于当前最小的卡方
        for i in range(len(freq)-1):
            v = chi22(freq[i:i+2])
            if minvalue is None or minvalue > v: # 小于当前最小卡方，更换最小值
                minvalue = v
                minidx = i
        
        # 如果最小卡方值小于阈值，则合并最小卡方值的相邻两组，并继续循环
        if (max_groups is not None and max_groups < len(freq)) or (threshold is not None and minvalue < threshold):
            # minidx 后一行合并到minidx
            tmp = freq[minidx] + freq[minidx+1]
            freq[minidx] = tmp
            # 删除minidx后一行
            freq = np.delete(freq,minidx+1,0)
            # 删除对应的切分点
            cutoffs = np.delete(cutoffs,minidx+1,0)
            
        else: # 最小卡方值不小于阈值，停止合并
            break
    return cutoffs

In [6]:
# bd_apply = bd.query('TRANSACTION_TYPE=="apply"').drop_duplicates(['ID_CARD_NO','CREATED_DATE'])

In [7]:
def tran(JSON,feature):
    try:
        return json.loads(JSON)[feature]
    except:
        print(JSON)
        return JSON

In [8]:
# bd_apply.reset_index(drop=True,inplace=True)

In [9]:
# for i in json.loads(bd_apply['EXPANDING'][0]).keys():
#     bd_apply[i] = bd_apply['EXPANDING'].map(lambda x:tran(x,i))

In [10]:
# bd_apply.to_csv(r'./merge_1st_ver.csv',index=False)

In [11]:
# import pandas_profiling
# report = pandas_profiling.ProfileReport(bd_apply)
# report.to_file('report.html')

In [12]:
# bd_apply =pd.read_csv(r'./merge_1st_ver.csv')

In [13]:
# bd_apply1 = bd_apply.drop_duplicates('ID_CARD_NO',keep='last')

In [14]:
# bd_tag_1 = pd.read_csv(r'./百度6月前表现.csv')
# bd_tag_2 = pd.read_csv(r'./百度7月180天表现.csv')
# bd_tag_3 = pd.read_csv(r'./百度8月150天表现.csv')
# bd_tag_1['Tt']=1
# bd_tag_2['Tt']=2
# bd_tag_3['Tt']=2

In [15]:
# bd_tag = pd.concat([bd_tag_1,bd_tag_2,bd_tag_3],axis=0)

In [16]:
# bd_tag.drop('cust_no',axis=1,inplace=True)

In [17]:
# bd_tag_avli = pd.DataFrame(bd_tag.groupby('cert_no')['ovdue_days'].max()).reset_index()

In [18]:
# bd_tag_avli.rename(columns={'cert_no':'ID_CARD_NO'},inplace=True)

In [19]:
# bd_avli = pd.merge(bd_apply1,bd_tag_avli,on='ID_CARD_NO',how='left')

In [20]:
# bd_tag_avli.shape

In [21]:
# bd_avli['ovdue_days'].fillna(-1,inplace=True)

In [22]:
# def tag(v):
#     if v>30:
#         return 1
#     elif v>0:
#         return 0.5
#     elif v==0:
#         return 0
#     else:
#         return -1
# bd_avli['tag'] = bd_avli['ovdue_days'].map(tag)

bd_avli 为输入整合后的数据表 base merge_with_fake_4

In [23]:
bd_avli = pd.read_csv(r'./merge_with_fake_4.csv')

E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
bd_avli['tag'].value_counts()

0.0    37144
1.0      915
Name: tag, dtype: int64

In [25]:
# bd_avli.query('CREATED_DATE<"2020-07-01"')['tag'].value_counts()

In [26]:
bd_avli['tag'].mean()

0.024041619590635593

In [27]:
# import pandas_profiling
# report = pandas_profiling.ProfileReport(bd_avli.query('CREATED_DATE<"2020-07-01"')[zx_feature])
# report.to_file('report.html')

In [28]:
bd_avli.query('CREATED_DATE>"2020-07-01"&CREATED_DATE<"2020-08-01"')['tag'].value_counts()

Series([], Name: tag, dtype: int64)

In [29]:
bd_avli.query('CREATED_DATE>"2020-08-01"')['tag'].value_counts()

Series([], Name: tag, dtype: int64)

In [30]:
# 数据处理

In [31]:
# zx_feature = [
#    'zxCreditCardDelqStatus30To60Days',
#    'zxHouseFundPayStatus',
#    'zxLoanRcyDelqDate',
#    'zxDomicile',
#    'zx1MthDelqMaxAmt',
#    'zxCreditCardAccuDelqCnt',
#    'zxCreditCardDelqStatus90To180Days',
#    'zxAccountCnt',
#    'zxRcy24MCreCardMaxDelqPeriod',
#    'zxHouseFundPayLastMonth',
#    'zxCreditCardDelqStatus30Dayyues',
#    'zxBadBebtsBalance',
#    'zxMaxMonths',
#    'zxMaxCreditAmt',
#    'zxLoanBalanceAmt',
#    'zxCommerHouseLoanBlance',
#    'zxGuaranteeCnt',
#    'zxMaxUpdated',
#    'zxRcy6MCreCardQueryCompanyCnt',
#    'zxCareer',
#    'zxLoanDelqStatus30Dayyues',
#    'zxFirstLoanMonth',
#    'zxBlanceShouldPayAmt',
#    'zxRcy12MCreCardMaxDelqPeriod',
#    'zxRestDisposalCnt',
#    'zxCreCardUseCreditRate',
#    'zxRcy3MCreCardQueryCnt',
#    'zxCreCardMaxMonthsCnt',
#    'zxRcy6MCreCardQueryCnt',
#    'zxCreCardStatus4',
#    'zxCreCardStatus5',
#    'zxCreCardStatus2',
#    'zxCreCardStatus3',
#    'zxPermanentAddress',
#    'zxLoanCompanyBalCnt',
#    'zxWarrantLoanRate1',
#    'zxFirstCreCardDate',
#    'zxCreCardStatus1',
#    'zxLoanRcyUpdateDate',
#    'zxRcy6MLoanQueryCnt',
#    'zxWarrantLoanType',
#    'zxCreCardDelqAccoutCnt',
#    'zxRcy24MAcctMaxDelqPeriod',
#    'zxIssuBankCnt',
#    'zxLoanDelqStatus6To12Months',
#    'zxDelqCnt',
#    'zxRcy1MLoanQueryCompanyCnt',
#    'zxCreCardNowShouldPayAmt',
#    'zxFirstLoanDate',
#    'zxLoanDelqStatus60To90Days',
#    'zxRcy1MCreCardQueryCnt',
#    'zxRcy6MAvgUseCreditAmt',
#    'zxCreditCardRcyUpdateDate',
#    'zxRcy24MCreCardTotalDelqCnt',
#    'zxEducation',
#    'zxRcyQueryDate',
#    'zxHouseFundCompanyRate',
#    'zxLoanDelqStatus90To180Days',
#    'zxHouseLoan1MAvgPayAmt',
#    'zxRcy3MCreCardQueryCompanyCnt',
#    'zxCreditCardDelqStatus6To12Months',
#    'zxBadBebtsCnt',
#    'zxPersonalLoanCnt',
#    'zxCreditBal',
#    'zxSex',
#    'zxAccountNowTotalDelqAmt',
#    'zxHouseFundMonthPayAmt',
#    'zxHouseFundSelfRate',
#    'zxLoanBalCnt',
#    'zxMinCreditAmt',
#    'zxTotalCreditAmt',
#    'zxMaritalStatus',
#    'zxRcy3MLoanQueryCompanyCnt',
#    'zxWarrantLoanCnt',
#    'zxCreditAvgLimit',
#    'zxCreditCardDelqStatus60To90Days',
#    'zxRcy6MLoanQueryCompanyCnt',
#    'zxRcy3MLoanQueryCnt',
#    'zxUseCreditAmt',
#    'zxCreCardAcctNowTotalDelqAmt',
#    'zxCreditRetCode',
#    'zxLoanDelqStatus30To60Days',
#    'zxRcy1MLoanQueryCnt',
#    'zxRcy1MCreCardQueryCompanyCnt',
#    'zxHouseFundRecordUpdate',
#    'zxWarrantLoanTypeUnnormal',
#    'zxHouseFundMonthShouldPayAmt',
#    'zxRcy6MCreCardMaxDelqPeriod',
#    'zxCreditCardDelqStatus24Months',
#    'zxRcy6MAvgPayAmt',
#    'zxCreCardDelqMaxAmt',
#    'zxCreditCardRcyDelqDate'
# ]

In [32]:
# zxCreditCardDelqStatus30To60Days   字典 不动 
# zxCreditCardDelqStatus30To60Days   zxHouseFundPayStatus# 

In [33]:
bd_avli_limit =bd_avli.query('tag==0|tag==1')

In [34]:
bd_avli_test = bd_avli_limit.copy()

In [35]:
zx_feature_enum = [
    'zxCreditCardDelqStatus30Dayyues',
    'zxCreditCardDelqStatus30To60Days',
    'zxCreditCardDelqStatus60To90Days',
    'zxCreditCardDelqStatus90To180Days',
    'zxCreditCardDelqStatus6To12Months',
    'zxCreditCardDelqStatus24Months',
    'zxLoanDelqStatus30Dayyues',# 留 0.01
    'zxLoanDelqStatus30To60Days',
    'zxLoanDelqStatus60To90Days',
    'zxLoanDelqStatus90To180Days',
    'zxLoanDelqStatus6To12Months',
    'zxCreCardStatus1',
    'zxCreCardStatus3',# 留0.01
    'zxCreCardStatus4',
    'zxCreCardStatus5',# 留0.01
    'zxHouseFundPayStatus',
    'zxCareer',
    'zxMaritalStatus',
    'zxEducation',
    'zxSex'
]
zx_feature_date = [
    'zxCreditCardRcyDelqDate',  #贷记卡字孔明偶从逾期日期  不留
    'zxCreditCardRcyUpdateDate', # 贷记卡记录最新更新时间   不留 
    'zxLoanRcyDelqDate',#贷款最近一次逾期日期  不留
#     'zxFirstLoanDate', # 首笔贷款获批日期 高度相关与 发放
    'zxFirstCreCardDate', # 首笔信用卡获批日期
    'zxFirstLoanMonth', #首笔贷款发放月份
#     'zxMaxUpdated',  #是信息最新更新日期  IV 太低 0.01
    'zxHouseFundPayLastMonth', # 缴至月份（住房公积金）
    'zxHouseFundRecordUpdate',
    'zxLoanRcyUpdateDate', # 贷款记录最新更新时间
]
zx_feature_date_str=[
    'zxRcy6MLoanQueryCnt',
    'zxRcy6MLoanQueryCompanyCnt',
#     'zxRcy6MCreCardQueryCompanyCnt', # IV_太低
#     'zxRcy6MCreCardQueryCnt', # IV_太低
    'zxRcy1MLoanQueryCnt',
    'zxRcy1MLoanQueryCompanyCnt',
#     'zxRcy1MCreCardQueryCompanyCnt',# IV_太低
#     'zxRcy1MCreCardQueryCnt', # IV_太低
    'zxRcy3MLoanQueryCompanyCnt',
    'zxRcy3MLoanQueryCnt',
#     'zxRcy3MCreCardQueryCompanyCnt', # IV_太低
#     'zxRcy3MCreCardQueryCnt', # IV_太低
    'zxAccountCnt',
    'zxTotalCreditAmt',
#     'zx1MthDelqMaxAmt',  合并后IV过低
    'zxIssuBankCnt',
    'zxUseCreditAmt',
    'zxCreditBal',
    'zxRcy6MAvgUseCreditAmt',
    'zxCreditAvgLimit',
    'zxCreCardUseCreditRate',
    'zxMaxCreditAmt',
    'zxMinCreditAmt',
    'zxBlanceShouldPayAmt',
    'zxCreCardNowShouldPayAmt',
    'zxHouseLoan1MAvgPayAmt',
#     'zxAccountNowTotalDelqAmt',  基本单一值
#     'zxDelqCnt', # IV 低
#     'zxWarrantLoanCnt', 基本单一值
    'zxRcy6MAvgPayAmt',
    'zxHouseFundMonthPayAmt',
    'zxHouseFundSelfRate',
    'zxHouseFundCompanyRate',
#     'zxCommerHouseLoanBlance',
#     'zxCreCardAcctNowTotalDelqAmt',
    'zxCreCardDelqMaxAmt',
    'zxCreCardMaxMonthsCnt',
    'zxCreCardDelqAccoutCnt',
#     'zxHouseFundMonthShouldPayAmt',
    'zxCreditCardAccuDelqCnt',
    'zxLoanBalCnt',
    'zxLoanBalanceAmt',
    'zxLoanCompanyBalCnt',
#     'zxPersonalLoanCnt',
    'zxRcy12MCreCardMaxDelqPeriod',
    'zxRcy24MCreCardMaxDelqPeriod',
    'zxRcy6MCreCardMaxDelqPeriod',
]

zx_feature_special=[
    'zxPermanentAddress',
    'zxDomicile'
]
zx_feature_drop=[
    'zxCreditRetCode',
    'zxRestDisposalCnt',  #信息量少
    'zxCreCardStatus2', # 
    'zxWarrantLoanTypeUnnormal',
    'zxWarrantLoanType', # 大部分是的未知
    'zxBadBebtsBalance', #  非未知值
    'zxMaxMonths', 
    'zxBadBebtsCnt',
    'zxRcyQueryDate', # 最近一次PBOC查询日期  留
]

In [36]:
zx_feature = zx_feature_enum+zx_feature_date+zx_feature_date_str

In [37]:
# 枚举类
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()

for i in ['zxCreditCardDelqStatus30Dayyues','zxCreditCardDelqStatus30To60Days','zxCreditCardDelqStatus60To90Days',
          'zxCreditCardDelqStatus90To180Days','zxCreditCardDelqStatus6To12Months','zxCreditCardDelqStatus24Months',
          'zxLoanDelqStatus30Dayyues','zxLoanDelqStatus30To60Days','zxLoanDelqStatus60To90Days',
          'zxLoanDelqStatus90To180Days','zxLoanDelqStatus6To12Months']:
    bd_avli_test[i] = lbl.fit_transform(bd_avli_test[i].astype(str))#将提示的包含错误数据类型这一列进行转换  
    print(lbl.classes_)

['0' '1' '10' '11' '2' '3' '4' '5' '6' '9']
['0' '1' '10' '11' '12' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '10' '11' '12' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '10' '11' '12' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '10' '11' '12' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '10' '11' '12' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '12' '2' '3' '4' '5' '6' '9']
['0' '1' '10' '11' '12' '13' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '10' '11' '12' '13' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '10' '11' '12' '13' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '10' '11' '12' '13' '2' '3' '4' '5' '6' '7' '8' '9']


In [38]:
for i in ['zxCreCardStatus1','zxCreCardStatus3','zxCreCardStatus4','zxCreCardStatus5','zxHouseFundPayStatus',
          'zxCareer','zxMaritalStatus','zxEducation','zxSex']:
    bd_avli_test[i] = lbl.fit_transform(bd_avli_test[i].astype(str))#将提示的包含错误数据类型这一列进行转换  
    print(lbl.classes_)

['0' '1' '2' '3' '5' '6']
['0' '1' '2' '3' '4' '5']
['0' '1' '2' '3' '5' '6']
['0' '1' '2' '3' '4' '5' '6']
['0' '1' '2' '3']
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '2' '3' '4']
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
['0' '1' '2']


In [39]:
print(IV.IV(bd_avli_test,'tag',zx_feature_enum))
# zxCreditCardDelqStatus30To60Days
# zxCreditCardDelqStatus24Months
# zxLoanDelqStatus90To180Days
# zxLoanDelqStatus6To12Months
# zxHouseFundPayStatus
# zxCreCardStatus5
# zxCareer
# zxMaritalStatus
# zxEducation

zxCreditCardDelqStatus30Dayyues		:0.781094501896974
zxCreditCardDelqStatus30To60Days		:0.5398086546354927
zxCreditCardDelqStatus60To90Days		:0.5333839304370908
zxCreditCardDelqStatus90To180Days		:0.5347789450410516
zxCreditCardDelqStatus6To12Months		:0.5409567922011805
zxCreditCardDelqStatus24Months		:0.5482141917501533
zxLoanDelqStatus30Dayyues		:0.5155717435352335
zxLoanDelqStatus30To60Days		:0.003976220575813837


E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars
E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


zxLoanDelqStatus60To90Days		:0.008173172289481983
zxLoanDelqStatus90To180Days		:0.004355480595070135
zxLoanDelqStatus6To12Months		:0.012585909006396827
zxCreCardStatus1		:0.8152036914550724
zxCreCardStatus3		:1.4710662625839852
zxCreCardStatus4		:0.49838325421682556
zxCreCardStatus5		:0.18322878873872925
zxHouseFundPayStatus		:0.18570361864722712
zxCareer		:0.01750712310488951
zxMaritalStatus		:0.1717659112269594
zxEducation		:1.5258439286488041
zxSex		:0.003973310964781272
            bad      good       WOE
zxSex                              
0      0.000000  0.000188      -inf
1      0.296175  0.267769  0.100827
2      0.703825  0.732043 -0.039309


In [40]:
pd.get_dummies(bd_avli_test,columns=zx_feature_enum)

,ID,CUST_NAME,ID_CARD_NO,MOBILE_NO,BANK_CARD_NO,CREATED_DATE,UPDATED_DATE,EXPANDING,TRANSACTION_TYPE,TRANSACTION_ID,...,zxEducation_3,zxEducation_4,zxEducation_5,zxEducation_6,zxEducation_7,zxEducation_8,zxEducation_9,zxSex_0,zxSex_1,zxSex_2
0,99c296b272d77d0b017300cc10382993,李加波,371329198911082714,13668690660,6217002290021005953,2020-06-29 23:57:20,2020-06-29 23:57:20,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,1160951258133627348,...,0,1,0,0,0,0,0,0,0,1
1,99c296b272d77d0b017300c59a692920,刘颜树,522728199509140663,13765441464,6228481198806286374,2020-06-29 23:50:16,2020-06-29 23:50:16,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,901896174999117926,...,0,1,0,0,0,0,0,0,1,0
2,99c296b272d77d0b017300bf767327e5,宁柳珍,430521198208165687,17363900837,6228481128475650972,2020-06-29 23:43:34,2020-06-29 23:43:34,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,1824236747107766050,...,0,1,0,0,0,0,0,0,1,0
3,99c296b172d77605017300ae84e02f5b,云佳琪,150103199401072143,15391141552,6217000410000146485,2020-06-29 23:25:04,2020-06-29 23:25:04,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,1079515297204273666,...,0,1,0,0,0,0,0,0,1,0
4,99c296b272d77d0b017300a965dc255e,张伟煌,440583198801063535,15915561358,6216607000010119355,2020-06-29 23:19:28,2020-06-29 23:19:28,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,975156991924536637,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38054,99c296b1723cb8a90172566186e51ca4,杨东,140427198711152414,15234530163,6212803600000732819,2020-05-27 21:45:31,2020-05-27 21:45:31,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,1420567212976606177,...,0,0,0,1,0,0,0,0,0,1
38055,99c296b2727a70390172a8ad31036be8,郭文芳,141033199308030063,14797400469,6221881600021297145,2020-06-12 21:17:02,2020-06-12 21:17:02,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,1325988575211528700,...,1,0,0,0,0,0,0,0,1,0
38056,99c296b1727a67fa0172a93d66f93b13,章穹麓,320524196502215811,13962142006,6228453018053804279,2020-06-12 23:54:32,2020-06-12 23:54:32,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,1307514927855486465,...,0,0,0,1,0,0,0,0,0,1
38057,99c296b2725680f501726157b4756d50,张培森,620102199401124317,15393157768,6215593700012630998,2020-05-30 00:50:37,2020-05-30 00:50:37,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,1251316314324528099,...,1,0,0,0,0,0,0,0,0,1


In [41]:
bd_avli_test1 = pd.get_dummies(bd_avli_test,columns=zx_feature_enum)

In [42]:
zx_feature_enum_new = list(set(bd_avli_test1.columns[:].to_list()) - set(bd_avli_test.columns[:].to_list()) )

In [43]:
zx_feature_enum_new.sort()

In [44]:
IV.IV(bd_avli_test1,'tag',zx_feature_date)

E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log
E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars


zxCreditCardRcyDelqDate		:0.2082882742928947
zxCreditCardRcyUpdateDate		:0.7161441447277286
zxLoanRcyDelqDate		:0.11192267619729858
zxFirstCreCardDate		:1.4075616294189164
zxFirstLoanMonth		:0.23636145503402198
zxHouseFundPayLastMonth		:0.185548504807676
zxHouseFundRecordUpdate		:0.1722328210402952
zxLoanRcyUpdateDate		:0.1733814638507929


,bad,good,WOE
zxLoanRcyUpdateDate,,,
-9999,0.000000,0.000888,-inf
224,0.000000,0.000081,-inf
225,0.000000,0.000027,-inf
226,0.000000,0.000081,-inf
227,0.000000,0.000135,-inf
...,...,...,...
3312,0.000000,0.000027,-inf
3512,0.000000,0.000027,-inf
3589,0.000000,0.000027,-inf


In [45]:
from pandas import DataFrame,Series
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
# X_train,X_test,y_train,y_test = train_test_split(bd_avli_limit[zx_feature],bd_avli_limit['tag'],test_size=0.3,random_state=2212333)

E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [46]:
#时间类特征处理

In [47]:
import math
def time_sub(v):
    if v=="-9999":
        return -9999
    else:
        try:
            return math.ceil((datetime.datetime.now()-datetime.datetime.strptime(v, "%Y-%m-%d %H:%M:%S")).days/365)
        except:
            try: 
                return math.ceil((datetime.datetime.now()-datetime.datetime.strptime(v, "%Y-%m-%d")).days/365)
            except:
                try:
                    return math.ceil((datetime.datetime.now()-datetime.datetime.strptime(str(v), "%Y%m%d")).days/365)
                except:
                    try:
                        return math.ceil((datetime.datetime.now()-datetime.datetime.strptime(v, "%Y/%m/%d")).days/365)
                    except:
                        try:
                            return math.ceil((datetime.datetime.now()-datetime.datetime.strptime(v, "%Y-%m")).days/365)
                        except:
                            print('err')
                            return -9999


In [48]:
# for i in zx_feature_date:
#     data = IV.IV(bd_avli_limit.query('CREATED_DATE<"2020-07-01"'),'tag',[i],True)
#     data.to_csv(i+'.csv')

In [49]:
def zxCreditCardRcyDelqDate_bin(v):
    if v==-9999:
        return 0
    elif v<=711:
        return 1
    else:
        return 2
    
def zxCreditCardRcyUpdateDate_bin(v):
    if v==-9999:
        return 0
    elif v<=803:
        return 1
    else:
        return 2
    
def zxLoanRcyDelqDate_bin(v):
    if v==-9999:
        return 0
    elif v<=436:
        return 1
    else:
        return 2

def zxFirstLoanMonth_bin(v):
    if v==-9999:
        return 0
    elif v<=46:
        return 1
    elif v<=78:
        return 2
    else:
        return 3

def zxFirstCreCardDate_bin(v):
    v_M = math.ceil(v/30)
    if v_M==-333:
        return 0
    elif v_M<=163:
        return 1
    else:
        return 2
    
def zxHouseFundPayLastMonth_bin(v):
    if v==-9999:
        return 0
    elif v<=34:
        return 1
    else:
        return 2
    
def zxHouseFundRecordUpdate_bin(v):
    if v==-9999:
        return 0
    elif v<=2841:
        return 1
    else:
        return 2
    
def zxLoanRcyUpdateDate_bin(v):
    if v==-9999:
        return 0
    elif v<=245:
        return 1
    elif v<=268:
        return 2
    elif v<=323:
        return 3
    elif v<=472:
        return 4
    else:
        return 5


In [50]:
bd_avli_test1['zxCreditCardRcyDelqDate_bin'] = bd_avli_test1['zxCreditCardRcyDelqDate'].map(zxCreditCardRcyDelqDate_bin)
bd_avli_test1['zxCreditCardRcyUpdateDate_bin'] = bd_avli_test1['zxCreditCardRcyUpdateDate'].map(zxCreditCardRcyUpdateDate_bin)
bd_avli_test1['zxLoanRcyDelqDate_bin'] = bd_avli_test1['zxLoanRcyDelqDate'].map(zxLoanRcyDelqDate_bin)
bd_avli_test1['zxFirstLoanMonth_bin'] = bd_avli_test1['zxFirstLoanMonth'].map(zxFirstLoanMonth_bin)
bd_avli_test1['zxFirstCreCardDate_bin'] = bd_avli_test1['zxFirstCreCardDate'].map(zxFirstCreCardDate_bin)
bd_avli_test1['zxHouseFundPayLastMonth_bin'] = bd_avli_test1['zxHouseFundPayLastMonth'].map(zxHouseFundPayLastMonth_bin)
bd_avli_test1['zxHouseFundRecordUpdate_bin'] = bd_avli_test1['zxHouseFundRecordUpdate'].map(zxHouseFundRecordUpdate_bin)
bd_avli_test1['zxLoanRcyUpdateDate_bin'] = bd_avli_test1['zxLoanRcyUpdateDate'].map(zxLoanRcyUpdateDate_bin)

In [51]:
# no_do = []
# cant_do = []
# for feature in zx_feature_date:
#     if bd_avli_test1[feature].value_counts().shape[0]>10000:
#         print(feature+' has so many value')
#         no_do.append(feature)
#         continue
#     try:
#         cutoffs = chiMerge(bd_avli_test1.query('CREATED_DATE<"2020-07-01"'),feature,"tag",10,3.8414588206941285)
#     except:
#         print(feature+' no long')
#         cant_do.append(feature)
#         bd_avli_test1[feature+'_bin'] =bd_avli_test1[feature]
#         continue
#     bd_avli_test1[feature+'_bin'] = 0
#     v = 0
#     for i in cutoffs:
#         v = v+1
#         bd_avli_test1.loc[bd_avli_test1[feature]>i,feature+'_bin'] = v
#     print(feature+' finish')

In [52]:
zx_feature_date_bin =[]
for i in zx_feature_date:
    zx_feature_date_bin.append(i+'_bin')

In [53]:
# bd_avli_test1['zxFirstCreCardDate_M'] = bd_avli_test1['zxFirstCreCardDate'].apply(lambda x:math.ceil(x/30))

In [54]:
# print(chiMerge(bd_avli_test1.query('CREATED_DATE<"2020-07-01"'),'zxLoanRcyUpdateDate',"tag",5,3.8414588206941285))

In [55]:
# print(chiMerge(bd_avli_limit.query('CREATED_DATE<"2020-07-01"'),'zxFirstLoanMonth',"tag",5,3.8414588206941285))

In [56]:
# bd_avli_limit.groupby(['zxFirstLoanMonth','tag'])['tag'].count()

In [57]:
IV.IV(bd_avli_test1.query('CREATED_DATE<"2020-07-01"'),'tag',zx_feature_date_bin,True)

                                  bad      good       WOE
zxCreditCardRcyDelqDate_bin                              
0                            0.781421  0.618996  0.233014
1                            0.116940  0.132350 -0.123788
2                            0.101639  0.248654 -0.894631
zxCreditCardRcyDelqDate_bin		:0.1712785905414528
                                    bad      good       WOE
zxCreditCardRcyUpdateDate_bin                              
0                              0.380328  0.079071  1.570692
1                              0.614208  0.909891 -0.392992
2                              0.005464  0.011038 -0.703086
zxCreditCardRcyUpdateDate_bin		:0.5933023494373287
                            bad      good       WOE
zxLoanRcyDelqDate_bin                              
0                      0.770492  0.799483 -0.036936
1                      0.088525  0.035484  0.914212
2                      0.140984  0.165033 -0.157504
zxLoanRcyDelqDate_bin		:0.05334954735591893
      

E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log
E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log
E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


,bad,good,WOE
zxLoanRcyUpdateDate_bin,,,
0,0.000000,0.000888,-inf
1,0.057923,0.097835,-0.524164
2,0.319126,0.344632,-0.076891
3,0.499454,0.397830,0.227490
4,0.104918,0.154776,-0.388800
5,0.018579,0.004038,1.526212


In [58]:
# bd_avli_limit['zxHouseFundSelfRate'] = bd_avli_limit['zxHouseFundSelfRate'].map(lambda x:int(x[:-1]))
# bd_avli_limit['zxHouseFundCompanyRate'] = bd_avli_limit['zxHouseFundCompanyRate'].map(lambda x:int(x[:-1]))

In [59]:
# no_do = []
# cant_do = []
# for feature in zx_feature_date_str:
#     if bd_avli_test1[feature].value_counts().shape[0]>4000:
#         print(feature+' has so many value')
#         no_do.append(feature)
#         continue
#     try:
#         cutoffs = chiMerge(bd_avli_test1.query('CREATED_DATE<"2020-07-01"'),feature,"tag",5,3.8414588206941285)
#     except:
#         print(feature+' no long')
#         cant_do.append(feature)
#         bd_avli_test1[feature+'_bin'] =bd_avli_test1[feature]
#         continue
#     bd_avli_test1[feature+'_bin'] = 0
#     v = 0
#     for i in cutoffs:
#         v = v+1
#         bd_avli_test1.loc[bd_avli_test1[feature]>0,feature+'_bin'] = v
#     print(feature+' finish')

In [61]:
#  chiMerge(bd_avli_test1.query('CREATED_DATE<"2020-07-01"'),'zxRcy6MLoanQueryCnt',"tag",5,3.8414588206941285)
# print(chiMerge(bd_avli_limit.query('CREATED_DATE<"2020-07-01"'),'zxLoanBalanceAmt',"tag",5,3.8414588206941285))

In [62]:
def zxAccountCnt_bin(v):
    if v==-9999:
        return 0
    elif v<=10:
        return 1
    else:
        return 2

def zxRcy1MLoanQueryCnt_bin(v):
    if v==0:
        return 0
    elif v<=4:
        return 1
    else:
        return 2

def zxRcy1MLoanQueryCompanyCnt_bin(v):
    if v==0:
        return 0
    elif v<=4:
        return 1
    else:
        return 2
    
def zxRcy6MLoanQueryCnt_bin(v):
    if v<=5:
        return 0
    elif v<=10:
        return 1
    else:
        return 2
    
def zxRcy6MLoanQueryCompanyCnt_bin(v):
    if v<=4:
        return 0
    elif v<=8:
        return 1
    else:
        return 2

def zxRcy3MLoanQueryCnt_bin(v):
    if v<=2:
        return 0
    elif v<=4:
        return 1
    else:
        return 2
    
def zxRcy3MLoanQueryCompanyCnt_bin(v):
    if v<=2:
        return 0
    elif v<=4:
        return 1
    else:
        return 2
    
def zxIssuBankCnt_bin(v):
    if v==-9999:
        return 0
    elif v==1:
        return 1
    else:
        return 2
    
def zxCreCardMaxMonthsCnt_bin(v):
    if v==-9999:
        return 0
    else:
        return 1

def zxCreCardDelqAccoutCnt_bin(v):
    if v==-9999:
        return 0
    else:
        return 1
    
def zxCreditCardAccuDelqCnt_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 1
    else:
        return 2
    
def zxLoanBalCnt_bin(v):
    if v==-9999:
        return 0
    elif v<=2:
        return 1
    elif v<=4:
        return 2
    else:
        return 3
    
def zxLoanCompanyBalCnt_bin(v):
    if v==-9999:
        return 0
    elif v<=3:
        return 1
    elif v<=7:
        return 2
    else:
        return 3
    
def zxRcy12MCreCardMaxDelqPeriod_bin(v):
    if v==-9999:
        return 0
    else:
        return 1

def zxRcy24MCreCardMaxDelqPeriod_bin(v):
    if v==-9999:
        return 0
    else:
        return 1
def zxHouseFundCompanyRate_bin(v):
    if v==0:
        return 0
    else:
        return 1
    
def zxHouseFundSelfRate_bin(v):
    if v==0:
        return 0
    else:
        return 1
    
def zxCreCardUseCreditRate_bin(v):
    if v==-9999:
        return 0
    elif v<=0.53:
        return 1
    else:
        return 2
    
def zxTotalCreditAmt_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 0
    else:
        return 1
    
def zxUseCreditAmt_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 0
    elif v<=131000:
        return 1
    else:
        return 2
    
def zxCreditBal_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 0
    elif v<=131000:
        return 1
    else:
        return 2
    
def zxRcy6MAvgUseCreditAmt_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 0
    else:
        return 1
    
def zxCreditAvgLimit_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 0
    elif v<=14000:
        return 1
    else:
        return 2
    
def zxMaxCreditAmt_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 0
    else:
        return 1
    
def zxMinCreditAmt_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 0
    else:
        return 1
    
def zxBlanceShouldPayAmt_bin(v):# 非单调
    if v<=1000:
        return 0
    elif v<=5000:
        return 1
    else:
        return 2
    
def zxCreCardNowShouldPayAmt_bin(v):
    if v==0:
        return 0
    elif v==-9999:
        return 0
    elif v<=7800:
        return 1
    else:
        return 2

def zxHouseLoan1MAvgPayAmt_bin(v):
    if v==-9999:
        return 0
    elif v==0:
        return 0
    elif v<=4600:
        return 1
    else:
        return 2
    
def zxRcy6MAvgPayAmt_bin(v):
    if v<=2800:
        return 0
    elif v<=6800:
        return 1
    else:
        return 2
    
def zxHouseFundMonthPayAmt_bin(v):
    if v<=0:
        return 0
    elif v<=700:
        return 1
    else:
        return 2

# def zxCommerHouseLoanBlance_bin(v):
    
# def zxCreCardAcctNowTotalDelqAmt_bin(v):
    
def zxCreCardDelqMaxAmt_bin(v):
    if v<=0:
        return 0
    elif v<=7000:
        return 1
    else:
        return 2

# def zxHouseFundMonthShouldPayAmt_bin(v):

def zxRcy6MCreCardMaxDelqPeriod_bin(v):
    if v==-9999:
        return 0
    else:
        return 1
    
def zxLoanBalanceAmt_bin(v):
    if v<=130000:
        return 0
    elif v<=1309000:
        return 1
    else:
        return 2

In [63]:
bd_avli_test1['zxAccountCnt_bin'] = bd_avli_test1['zxAccountCnt'].map(zxAccountCnt_bin)
bd_avli_test1['zxRcy1MLoanQueryCnt_bin'] = bd_avli_test1['zxRcy1MLoanQueryCnt'].map(zxRcy1MLoanQueryCnt_bin)
bd_avli_test1['zxRcy1MLoanQueryCompanyCnt_bin'] = bd_avli_test1['zxRcy1MLoanQueryCompanyCnt'].map(zxRcy1MLoanQueryCompanyCnt_bin)
bd_avli_test1['zxRcy6MLoanQueryCnt_bin'] = bd_avli_test1['zxRcy6MLoanQueryCnt'].map(zxRcy6MLoanQueryCnt_bin)
bd_avli_test1['zxRcy6MLoanQueryCompanyCnt_bin'] = bd_avli_test1['zxRcy6MLoanQueryCompanyCnt'].map(zxRcy6MLoanQueryCompanyCnt_bin)
bd_avli_test1['zxRcy3MLoanQueryCnt_bin'] = bd_avli_test1['zxRcy3MLoanQueryCnt'].map(zxRcy3MLoanQueryCnt_bin)
bd_avli_test1['zxRcy3MLoanQueryCompanyCnt_bin'] = bd_avli_test1['zxRcy3MLoanQueryCompanyCnt'].map(zxRcy3MLoanQueryCompanyCnt_bin)
bd_avli_test1['zxIssuBankCnt_bin'] = bd_avli_test1['zxIssuBankCnt'].map(zxIssuBankCnt_bin)
bd_avli_test1['zxCreCardMaxMonthsCnt_bin'] = bd_avli_test1['zxCreCardMaxMonthsCnt'].map(zxCreCardMaxMonthsCnt_bin)
bd_avli_test1['zxCreCardDelqAccoutCnt_bin'] = bd_avli_test1['zxCreCardDelqAccoutCnt'].map(zxCreCardDelqAccoutCnt_bin)
bd_avli_test1['zxCreditCardAccuDelqCnt_bin'] = bd_avli_test1['zxCreditCardAccuDelqCnt'].map(zxCreditCardAccuDelqCnt_bin)
bd_avli_test1['zxLoanBalCnt_bin'] = bd_avli_test1['zxLoanBalCnt'].map(zxLoanBalCnt_bin)
bd_avli_test1['zxLoanCompanyBalCnt_bin'] = bd_avli_test1['zxLoanCompanyBalCnt'].map(zxLoanCompanyBalCnt_bin)
bd_avli_test1['zxRcy12MCreCardMaxDelqPeriod_bin'] = bd_avli_test1['zxRcy12MCreCardMaxDelqPeriod'].map(zxRcy12MCreCardMaxDelqPeriod_bin)
bd_avli_test1['zxRcy24MCreCardMaxDelqPeriod_bin'] = bd_avli_test1['zxRcy24MCreCardMaxDelqPeriod'].map(zxRcy24MCreCardMaxDelqPeriod_bin)
bd_avli_test1['zxRcy6MCreCardMaxDelqPeriod_bin'] = bd_avli_test1['zxRcy6MCreCardMaxDelqPeriod'].map(zxRcy6MCreCardMaxDelqPeriod_bin)

bd_avli_test1['zxHouseFundCompanyRate_bin'] = bd_avli_test1['zxHouseFundCompanyRate'].map(zxHouseFundCompanyRate_bin)
bd_avli_test1['zxHouseFundSelfRate_bin'] = bd_avli_test1['zxHouseFundSelfRate'].map(zxHouseFundSelfRate_bin)
bd_avli_test1['zxCreCardUseCreditRate_bin'] = bd_avli_test1['zxCreCardUseCreditRate'].map(zxCreCardUseCreditRate_bin)
bd_avli_test1['zxTotalCreditAmt_bin'] = bd_avli_test1['zxTotalCreditAmt'].map(zxTotalCreditAmt_bin)
bd_avli_test1['zxUseCreditAmt_bin'] = bd_avli_test1['zxUseCreditAmt'].map(zxUseCreditAmt_bin)
bd_avli_test1['zxCreditBal_bin'] = bd_avli_test1['zxCreditBal'].map(zxCreditBal_bin)
bd_avli_test1['zxRcy6MAvgUseCreditAmt_bin'] = bd_avli_test1['zxRcy6MAvgUseCreditAmt'].map(zxRcy6MAvgUseCreditAmt_bin)
bd_avli_test1['zxCreditAvgLimit_bin'] = bd_avli_test1['zxCreditAvgLimit'].map(zxCreditAvgLimit_bin)
bd_avli_test1['zxMaxCreditAmt_bin'] = bd_avli_test1['zxMaxCreditAmt'].map(zxMaxCreditAmt_bin)
bd_avli_test1['zxMinCreditAmt_bin'] = bd_avli_test1['zxMinCreditAmt'].map(zxMinCreditAmt_bin)
bd_avli_test1['zxBlanceShouldPayAmt_bin'] = bd_avli_test1['zxBlanceShouldPayAmt'].map(zxBlanceShouldPayAmt_bin)
bd_avli_test1['zxCreCardNowShouldPayAmt_bin'] = bd_avli_test1['zxCreCardNowShouldPayAmt'].map(zxCreCardNowShouldPayAmt_bin)
bd_avli_test1['zxHouseLoan1MAvgPayAmt_bin'] = bd_avli_test1['zxHouseLoan1MAvgPayAmt'].map(zxHouseLoan1MAvgPayAmt_bin)
bd_avli_test1['zxRcy6MAvgPayAmt_bin'] = bd_avli_test1['zxRcy6MAvgPayAmt'].map(zxRcy6MAvgPayAmt_bin)
bd_avli_test1['zxHouseFundMonthPayAmt_bin'] = bd_avli_test1['zxHouseFundMonthPayAmt'].map(zxHouseFundMonthPayAmt_bin)
bd_avli_test1['zxCreCardDelqMaxAmt_bin'] = bd_avli_test1['zxCreCardDelqMaxAmt'].map(zxCreCardDelqMaxAmt_bin)
bd_avli_test1['zxLoanBalanceAmt_bin'] = bd_avli_test1['zxLoanBalanceAmt'].map(zxLoanBalanceAmt_bin)

In [64]:
zx_feature_date_str_bin =[]
for i in zx_feature_date_str:
    zx_feature_date_str_bin.append(i+'_bin')

In [65]:
# for i in [
# #     'zxTotalCreditAmt'
# #     ,'zx1MthDelqMaxAmt'   
# #     ,'zxUseCreditAmt'
# #     ,'zxCreditBal'
# #     ,'zxRcy6MAvgUseCreditAmt'
# #     ,'zxCreditAvgLimit'
# #     ,'zxMaxCreditAmt'
# #     ,'zxMinCreditAmt'
# #     ,'zxBlanceShouldPayAmt'
# #     ,'zxCreCardNowShouldPayAmt'
# #     ,'zxHouseLoan1MAvgPayAmt'
# #     ,'zxRcy6MAvgPayAmt'
# #     ,'zxHouseFundMonthPayAmt'
# #     ,'zxCommerHouseLoanBlance'
# #     ,'zxCreCardAcctNowTotalDelqAmt'
# #     ,'zxCreCardDelqMaxAmt'
# #     ,'zxHouseFundMonthShouldPayAmt'
# 'zxLoanBalanceAmt'
# ]:
#     data = IV.IV(bd_avli_limit.query('CREATED_DATE<"2020-07-01"'),'tag',[i],True)
#     data.to_csv('z_'+i+'.csv')

In [66]:
# def AMT_fitst_bin(v):
#     if v==-9999:
#         return 0
#     if v<=10000:
#         return math.ceil(v/100)*100
#     if v>10000:
#         return math.ceil(v/1000)*1000

In [67]:
# def Use_Rate_bin(v):
#     if v==-9999:
#         return -9999
#     else:
#         return math.ceil(v*100)/100

In [68]:
# for i in [
# #     'zxTotalCreditAmt'
# #     ,'zx1MthDelqMaxAmt'   
# #     ,'zxUseCreditAmt'
# #     ,'zxCreditBal'
# #     ,'zxRcy6MAvgUseCreditAmt'
# #     ,'zxCreditAvgLimit'
# #     ,'zxMaxCreditAmt'
# #     ,'zxMinCreditAmt'
# #     ,'zxBlanceShouldPayAmt'
# #     ,'zxCreCardNowShouldPayAmt'
# #     ,'zxHouseLoan1MAvgPayAmt'
# #     ,'zxRcy6MAvgPayAmt'
# #     ,'zxHouseFundMonthPayAmt'
# #     ,'zxCommerHouseLoanBlance'
# #     ,'zxCreCardAcctNowTotalDelqAmt'
# #     ,'zxCreCardDelqMaxAmt'
# #     ,'zxHouseFundMonthShouldPayAmt'
# 'zxLoanBalanceAmt'
# ]:
#     if i != "zxCreCardUseCreditRate":
#         bd_avli_limit[i] = bd_avli_limit[i].map(AMT_fitst_bin)
#     if i == "zxCreCardUseCreditRate":
#         bd_avli_limit[i] = bd_avli_limit[i].map(Use_Rate_bin)

In [69]:
# bd_avli_limit['zxCreCardUseCreditRate'] = bd_avli_limit['zxCreCardUseCreditRate'].map(Use_Rate_bin)

In [70]:
# bd_avli_limit['zxTotalCreditAmt'].value_counts()

In [71]:
zx_feature = zx_feature_enum_new+zx_feature_date_bin+zx_feature_date_str_bin
X_train,X_test,y_train,y_test = train_test_split(bd_avli_test1[zx_feature],bd_avli_test1['tag'],test_size=0.3,random_state=2212333)

In [60]:
X_train = bd_avli_limit.query('CREATED_DATE<"2020-07-01"')

In [61]:
X_train['tag'].value_counts()

0.0    37144
1.0      544
Name: tag, dtype: int64

In [62]:
X_train_good = X_train.query('tag==0').sample(10000,random_state=10000)

In [63]:
X_train_bad = X_train.query('tag==1')

In [64]:
X_train_limit = pd.concat([X_train_bad,X_train_good],axis=0)

In [65]:
X_test = bd_avli_limit.query('CREATED_DATE>"2020-07-01"')

In [66]:
from sklearn.linear_model import LogisticRegression,SGDClassifier

LR= LogisticRegression(penalty='l2', dual=False,tol=0.001, C=1, fit_intercept=True, intercept_scaling=1,
                       class_weight= None, random_state=None,  
                       max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)
SGD = SGDClassifier(loss='log')


LR.fit(X_train_limit[zx_feature_date_change+zx_feature_date_str_bin+zx_feature_date_change_bin],X_train_limit['tag'])
SGD.fit(X_train_limit[zx_feature_date_change],X_train_limit['tag'])

E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


SGDClassifier(loss='log')

In [67]:
final_pre1 = LR.predict_log_proba(X_test[zx_feature_date_change+zx_feature_date_str_bin+zx_feature_date_change_bin])[:,1]

ValueError: Found array with 0 sample(s) (shape=(0, 53)) while a minimum of 1 is required.

In [68]:
pd.DataFrame(final_pre1,columns=['predict'])['predict'].value_counts()

NameError: name 'final_pre1' is not defined

In [69]:

    label_value = X_test['tag']
#     print('f1_score: ', f1_score(label_value, final_pre1))

    print('auc: ', roc_auc_score(label_value, final_pre1))

#     print('accuracy_scorey: ',
#           accuracy_score(label_value, final_pre1))

#     print('recall_score: ',
#           recall_score(label_value, final_pre1))
#     print('recall',help_df.query('predict==1')['decrease2'].sum()/help_df['decrease2'].sum())

NameError: name 'final_pre1' is not defined

In [145]:
final_pre1.sum()

-146515.08342934397

In [166]:
final_pre1.max()

0.19636895062699325

In [124]:
final_pre2 = final_pre1*5

In [125]:
final_pre2.sum()

19201.72868305496

In [127]:
final_pre2

array([0.23774014, 0.27084467, 0.49394937, ..., 0.59268959, 0.86755189,
       0.54851522])

In [130]:
s = pd.DataFrame(final_pre2,columns=['predict'])

In [133]:
s['predict2'] = s['predict'].apply(lambda x:1 if x>=0.4 else 0)

In [134]:
s['predict2'].sum()

23902

In [167]:
X_test['predict']=final_pre1

E:\Users\zhengkw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [168]:
X_test.query('tag==1')[['predict','tag']]

,predict,tag
83404,0.082320,1.0
83566,0.107613,1.0
83724,0.073610,1.0
83747,0.076759,1.0
83793,0.078526,1.0
...,...,...
179981,0.061080,1.0
180431,0.158353,1.0
180712,0.135670,1.0
180773,0.131198,1.0


In [170]:
X_test.query('predict>0.078526')

,ID,CUST_NAME,ID_CARD_NO,MOBILE_NO,BANK_CARD_NO,CREATED_DATE,UPDATED_DATE,EXPANDING,TRANSACTION_TYPE,TRANSACTION_ID,...,zxLoanRcyDelqDate_change,zxRcyQueryDate_change,zxFirstLoanDate_change,zxFirstCreCardDate_change,zxFirstLoanMonth_change,zxMaxUpdated_change,zxHouseFundPayLastMonth_change,zxHouseFundRecordUpdate_change,zxLoanRcyUpdateDate_change,predict
83208,99c296b172d7760501730801c45d774b,白建涛,511133199708262810,13219670826,6228410334585738170,2020-07-01 09:33:20,2020-07-01 09:33:20,"{""zxCreditCardDelqStatus30To60Days"":""-9999"",""z...",apply,1135673425765390188,...,-9999,217,681,-9999,24,-9999,-9999,-9999,230,0.098790
83212,99c296b272d77d0b017307e7577912f7,包志刚,411302198702020018,15203816128,6226632201313876,2020-07-01 09:04:28,2020-07-01 09:04:28,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,1186791413431086436,...,308,217,1159,1953,39,237,-9999,-9999,236,0.108527
83218,99c296b172d776050173084463b10f74,苏尚杰,330327199508212498,15967767763,6217734600594626,2020-07-01 10:46:06,2020-07-01 10:46:06,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,997148094156734176,...,-9999,219,1397,1712,47,378,-9999,-9999,227,0.082234
83219,99c296b172d776050173081a6b0e0041,付立鹏,430525199904060037,15774061988,6217002930106916554,2020-07-01 10:00:15,2020-07-01 10:00:15,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,947198308686227377,...,-9999,242,1007,332,34,332,-9999,-9999,255,0.082582
83220,99c296b172d776050173081c7406007b,楚子良,130530198505161514,13731130848,6217991430003665225,2020-07-01 10:02:29,2020-07-01 10:02:29,"{""zxCreditCardDelqStatus30To60Days"":""-9999"",""z...",apply,951627078263237346,...,-9999,217,1480,-9999,50,-9999,-9999,-9999,248,0.087049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181239,99c296b173f7ddb301742b3f92102011,干传争,340123198401111911,17715300144,6222021103012286671,2020-08-26 22:50:20,2020-08-26 22:50:20,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,4610007235995629803,...,-9999,160,1833,1325,62,247,-9999,-9999,175,0.133641
181240,99c296b173f7ddb301742dce0ef509a5,徐安娜,520102198111023827,18798781484,6222082402002568718,2020-08-27 10:45:13,2020-08-27 10:45:13,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,4610007717031961900,...,978,160,4265,5613,143,193,-9999,-9999,163,0.080816
181246,99c296b173f7ddb301742dd9fa11263a,王加飞,330183198711104316,13588783430,6217730800421917,2020-08-27 10:58:14,2020-08-27 10:58:14,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,4610007717031965939,...,1070,160,2127,4541,72,625,-9999,-9999,172,0.118538
181253,99c296b173f7ddb3017429fd22f96cfc,杨帅,513701199404141810,18384580806,6217251400022877007,2020-08-26 16:58:09,2020-08-26 16:58:09,"{""zxCreditCardDelqStatus30To60Days"":""N"",""zxHou...",apply,4609998027585728168,...,308,160,1405,931,48,205,-9999,-9999,170,0.173510


In [173]:
X_test.loc[83793].tag()

ID                                99c296b272d77d0b0173079f546d79c5
CUST_NAME                                                      曾丽妹
ID_CARD_NO                                      460028199311242449
MOBILE_NO                                              18889492199
BANK_CARD_NO                                   6215582201007560228
                                                ...               
zxMaxUpdated_change                                            308
zxHouseFundPayLastMonth_change                                  97
zxHouseFundRecordUpdate_change                                 278
zxLoanRcyUpdateDate_change                                     228
predict                                                  0.0785259
Name: 83793, Length: 138, dtype: object